# Train OCR text Detector quick example

In [1]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')


sys.path.append(NOMEROFF_NET_DIR)

from NomeroffNet.TextDetectors.base import OCR

In [2]:
# auto download latest dataset
from NomeroffNet.tools import modelhub

# auto download latest dataset
info = modelhub.download_dataset_for_model("By")
PATH_TO_DATASET = info["dataset_path"]

# local path dataset
#PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "./data/dataset/OptionsDetector/numberplate_options_example")

In [3]:
PATH_TO_DATASET

'/mnt/data/var/www/nomeroff-net/NomeroffNet/tools/../../data/./dataset/TextDetector/By/autoriaNumberplateOcrBy-2020-10-09'

In [4]:
DATASET_NAME = "by"
VERSION = "2021_07_13_pytorch_lightning"

RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, 
                                 "models/", 
                                 'anpr_ocr_{}_{}.ckpt'.format(DATASET_NAME, VERSION))

In [5]:
RESULT_MODEL_PATH

'/mnt/data/var/www/nomeroff-net/models/anpr_ocr_by_2021_07_13_pytorch_lightning.ckpt'

In [6]:
class by(OCR):
    def __init__(self):
        OCR.__init__(self)
        # only for usage model
        # in train generate automaticly
        self.letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "E", "H", "I", "K", "M", "O", "P", "T", "X"]
        

        # Train hyperparameters
        self.batch_size = 32
        self.epochs = 1
        self.gpus = 0
        
        self.height = 64
        self.width = 128

In [7]:
ocrTextDetector = by()
model = ocrTextDetector.prepare(PATH_TO_DATASET, use_aug=False)

GET ALPHABET
Max plate length in "val": 8
Max plate length in "train": 8
Max plate length in "test": 8
Letters train  {'K', 'P', '0', 'H', 'X', 'I', '2', 'B', '4', '3', '7', '5', '6', '9', '1', 'T', 'M', '8', 'A', 'E', 'O', 'C'}
Letters val  {'K', 'P', '0', 'H', 'X', 'I', '2', 'B', '4', '3', '7', '5', '6', '9', '1', 'T', 'M', '8', 'A', 'E', 'O', 'C'}
Letters test  {'K', 'P', '0', 'H', 'X', 'I', '2', 'B', '4', '3', '7', '5', '6', '9', '1', 'T', 'M', '8', 'A', 'E', 'O', 'C'}
Max plate length in train, test and val do match
Letters in train, val and test do match
Letters: 0 1 2 3 4 5 6 7 8 9 A B C E H I K M O P T X

EXPLAIN DATA TRANSFORMATIONS
START BUILD DATA
DATA PREPARED


In [8]:
model = ocrTextDetector.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


RuntimeError: input_lengths must be of size batch_size

In [ ]:
len({'K', '1', '0', 'X', 'A', 'C', 'E', 'H', 'M', 'O', '2', 'P', '4', '9', 'T', '3', 'B', '7', 'I', '8', '6', '5'})

In [ ]:
ocrTextDetector.test(verbose=True)

In [ ]:
from NomeroffNet.data_modules.data_loaders import TextImageGenerator

In [ ]:
dataset = TextImageGenerator(os.path.join(PATH_TO_DATASET, "val"), ocrTextDetector.letters, 8)

In [ ]:
(x, input_lengths, target_lengths), label = dataset[1]

In [ ]:
x

In [ ]:
input_lengths

In [ ]:
target_lengths

In [ ]:
label

In [ ]:
128//(2*2) -2